# Intermediate Joins in SQL Using Chinook Database
The goal for this notebook is to continue practicing SQL joins using a database with more than two tables in it. This will allow for some interesting queries.

<img src = 'chinook-schema.svg' width = 600>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

First Query:

1. The id of the track, track_id.
2. The name of the track, track_name.
3. The name of media type of the track, track_type.
4. The price that the customer paid for the track, unit_price.
5. The quantity of the track that was purchased, quantity.

Based on the schema above, this will involve three separate tables:

1. Track
2. Invoice Line
3. Media Type


In [6]:
%%sql
SELECT tr.track_id, 
       tr.name track_name, 
       mt.name AS media_type, 
       tr.unit_price, 
       il.quantity
FROM invoice_line il
INNER JOIN track tr ON tr.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = tr.media_type_id
WHERE il.invoice_id = 4;

 * sqlite:///chinook.db
Done.


track_id,track_name,media_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",Protected AAC audio file,0.99,1
2560,Violent Pornography,MPEG audio file,0.99,1
3336,War Pigs,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,MPEG audio file,0.99,1
1872,Attitude,MPEG audio file,0.99,1
748,Dealer,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
2514,Spoonman,MPEG audio file,0.99,1


Second Query:

Add a column containing the artists name to the previous query.

1. The column should be called artist_name
2. The column should be placed between track_name and track_type

To get this single extra column, I will have to add two joins. 

1. Track table joins Album table on album_id
2. Album table joins Artist table on artist_id

Then, I will get Artist name from the name value of the Artist table


In [7]:
%%sql
SELECT tr.track_id, 
       tr.name track_name, 
       ar.name artist_name, 
       mt.name AS media_type, 
       tr.unit_price, 
       il.quantity
FROM invoice_line il
INNER JOIN track tr ON tr.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = tr.media_type_id
INNER JOIN album al ON al.album_id = tr.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
WHERE il.invoice_id = 4;

 * sqlite:///chinook.db
Done.


track_id,track_name,artist_name,media_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",The King's Singers,Protected AAC audio file,0.99,1
2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
1872,Attitude,Metallica,MPEG audio file,0.99,1
748,Dealer,Deep Purple,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


Third Query:

Write a query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased. Your query should be sorted from most tracks purchased to least tracks purchased and return the following columns, in order:

1. album, the title of the album
2. artist, the artist who produced the album
3. tracks_purchased the total number of tracks purchased from that album

Tables Involved:
1. album  artist.artist_id = album.artist_id and album.album_id = track.album_id
2. artist (name)
2. track (track_id) track.track_id = invoice_line.track_id
3. invoice line


In [14]:
%%sql
SELECT ta.album_name album,
       ta.artist_name artist,
       COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (SELECT al.title album_name,
                   ar.name artist_name,
                   tr.track_id track_id
            FROM track tr
            INNER JOIN album al ON tr.album_id = al.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
            ) ta
ON ta.track_id = il.track_id
GROUP BY album
ORDER BY tracks_purchased DESC
LIMIT 5;

 * sqlite:///chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


## Recursive Joins

Write a query that returns information about each employee and their supervisor.

The report should include employees even if they do not report to another employee.

The report should be sorted alphabetically by the employee_name column.

Your query should return the following columns, in order:

1. employee_name - containing the first_name and last_name columns separated by a space, eg Luke Skywalker
2. employee_title - the title of that employee
3. supervisor_name - the first and last name of the person the employee reports to, in the same format as employee_name
4. supervisor_title - the title of the person the employee reports to


Tables that will be involved for this:

Only the *employee* table


In [18]:
%%sql
SELECT e1.first_name || ' ' || e1.last_name AS employee_name,
       e1.title employee_title,
       e2.first_name || ' ' || e2.last_name AS supervisor_name,
       e2.title supervisor_title
FROM employee e1
LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
ORDER BY employee_name;

 * sqlite:///chinook.db
Done.


employee_name,employee_title,supervisor_name,supervisor_title
Andrew Adams,General Manager,None,None
Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
Laura Callahan,IT Staff,Michael Mitchell,IT Manager
Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
Michael Mitchell,IT Manager,Andrew Adams,General Manager
Nancy Edwards,Sales Manager,Andrew Adams,General Manager
Robert King,IT Staff,Michael Mitchell,IT Manager
Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


## Pattern Matching in SQL

### Problem:
You have just returned from lunch to see another phone message on your desk: "Call Belle."

Write a query that finds the contact details of a customer with a first_name containing Belle from the database. Your query should include the following columns, in order:

1. first_name
2. last_name
3. phone

### Solution 
This is going to involve the use of the customer table only, with the columns titled as above.

In [23]:
%%sql
SELECT first_name,
       last_name,
       phone
FROM customer
WHERE first_name LIKE "%Belle%";

 * sqlite:///chinook.db
Done.


first_name,last_name,phone
Isabelle,Mercier,+33 03 80 73 66 99


<img src = 'chinook-schema.svg' width = 600>

## Categorization of Results

### Problem

Write a query that summarizes the purchases of each customer. For the purposes of this exercise, we do not have any two customers with the same name.

Your query should include the following columns, in order:
1. customer_name - containing the first_name and last_name columns separated by a space, eg Luke Skywalker.
2. number_of_purchases, counting the number of purchases made by each customer.
3. total_spent - the total sum of money spent by each customer.
4. customer_category - a column that categorizes the customer based on their total purchases. The column should contain the following values:  
    4.1 small spender - If the customer's total purchases are less than \\$40.  
    4.2 big spender - If the customer's total purchases are greater than \\$100.    
    4.3 regular - If the customer's total purchases are between \\$40 and \\$100 (inclusive).       
5. Order your results by the customer_name column.

### Solution

**Tables Required**
1. customer linked to invoice on customer.customer_id = invoice.customer_id
2. invoice

In [60]:
%%sql

SELECT 
    c.first_name || ' ' || c.last_name customer_name,
    COUNT(i.invoice_id) number_of_purchases,
    SUM(i.total) total_spent,
    CASE 
    WHEN SUM(i.total) < 40 THEN 'small spender'
    WHEN SUM(i.total) > 100 THEN 'big spender'
    ELSE 'regular'
    END AS customer_category
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
GROUP BY customer_name
ORDER BY customer_name
LIMIT 10; -- Just want to shorten the output

 * sqlite:///chinook.db
Done.


customer_name,number_of_purchases,total_spent,customer_category
Aaron Mitchell,8,70.28999999999999,regular
Alexandre Rocha,10,69.3,regular
Astrid Gruber,9,69.3,regular
Bjørn Hansen,9,72.27000000000001,regular
Camille Bernard,9,79.2,regular
Daan Peeters,7,60.38999999999999,regular
Dan Miller,12,95.03999999999999,regular
Diego Gutiérrez,5,39.6,small spender
Dominique Lefebvre,9,72.27,regular
Eduardo Martins,12,60.39,regular
